tensorflow는 저장된 모델 `SavedModel`(model을 build하는데 쓰였던 코드가 없이도 실행시킬 수 있게 해준다)을 검사할 수 있는 `saved_model_cli` 명령어가 있다.    
[TensorFlow Saved Model](https://www.tensorflow.org/guide/saved_model?hl=ko)    


1. SavedModel에서 사용 가능한 모든 MetaGraphDef 태그 세트 (메타그래프 하나 저장)
```bash
!saved_model_cli show --dir {model_path}
```


2. MetaGraphDef에서 사용 가능한 모든 SignatureDef 키 (meta graph , signaturedef key save)
```bash
!saved_model_cli show --dir {model_path} --tag_set serve
```

3. 특정 SignatureDef에 대한 모든 입력 및 출력 텐서 정보(TensorInfo) (signature, input/output name, type, shape save) SignatureDef 키를 signature_def 옵션으로 전달    
계산 그래프를 실행하기 위해 입력 텐서의 텐서 키 값, 크기 및 데이터 타입을 알고자 할 때 매우 유용
```bash
!saved_model_cli show --dir {model_path} --tag_set serve --signature_def serving_default
```

4. SavedModel에 사용 가능한 모든 정보를 표시
```bash
!saved_model_cli show --dir {model_path} --all
```
---
Serving에서 사용되는 그래프 : input, output = signature

- Cassification SignatureDef : 분류 모델에 최적화되어 클래스 종류나 클래스별 정확도등을 옵션으로 가짐
- Predict SignatureDef : 분류 모델뿐 아니라 모든 모델에 범용적으로사용될 수 있는 형태

# TensorFlow를 이용한 서빙
TF serving install 방법 
1. Docker 이미지 사용
2. System package manager 사용
3. 소스에서 설치

### 1. 도커를 이용하는 코드이다. 
```Shell Script
# TF 서빙 도커 이미지 다운로드
docker pull tensorflow/serving

# 이미지 실행위한 도커 컨테이너 만들기 
export ML_PATH=$HOME/ml # or wherever this project is
docker run -it --rm -p 8500:8500 -p 8501:8501 \
   -v "$ML_PATH/my_mnist_model:/models/my_mnist_model" \
   -e MODEL_NAME=my_mnist_model \
   tensorflow/serving
```
아래 코드로 서버를 running 할 수 있다.

In [ ]:
os.environ["MODEL_DIR"] = os.path.split(os.path.abspath(model_path))[0]
%%bash --bg
nohup tensorflow_model_server \
     --rest_api_port=8501 \
     --model_name=my_mnist_model \
     --model_base_path="${MODEL_DIR}" >server.log 2>&1
!tail server.log

### 2. 아래는 REST API로 TF 서빙에 쿼리 하는 방법이다.
- input, output의 크기가 거대하지만 않으면 ㄱㅊ다.
- JSON기반 -> 텍스트 사용 -> 매우 클 수 있음
- 직렬화, 역직렬화 진행한다는게 비효율적임

In [ ]:
import json
# 필요한 시그니처 이름, 입력 데이터 
input_data_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(), # json은 text로 저장하므로 np는 list로 저장해야한다.
})

import requests

# HTTP POST method로 TF 서빙을 전송한다. requests 라이브러리 사용.
SERVER_URL = 'http://localhost:8501/v1/models/my_mnist_model:predict'
response = requests.post(SERVER_URL, data=input_data_json)
response.raise_for_status() # raise an exception in case of error
response = response.json()
# response.keys() = dict_keys(['predictions'])

y_proba = np.array(response["predictions"]) # list로 변경했었기 때문에 np로 다시 변경해준다. 소수점 반올림.

### 3. gRPC API로 TF 서빙에 쿼리하는 방법
gRPC API의 **input**으로는 '직렬화된 Predict Request 프로토콜 버퍼'임    
**output** : PredictResponse Protocol buffer
1. request 만들기

In [ ]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest

request = PredictRequest() # predictRequest protocol buffer 생성 
# request 필드 채우기 
request.model_spec.name = model_name # model name field
request.model_spec.signature_name = "serving_default" # 호출할 signature_name field
input_name = model.input_names[0]  # Tensor protocol buffer 타입의 input data
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))
# tf.make_tensor_proto(X_new) : tensor, np 기반으로 tensor protocol buffer 생성 

2. 서버로 request 보내고 response 받기

In [ ]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel('localhost:8500') # gRPC 통신 채널 생성
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel) 
response = predict_service.Predict(request, timeout=10.0) # 10초의 통신 시간 여유

3. PredictResponse protocol buffer -변경-> Tensor

In [ ]:
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
y_proba = tf.make_ndarray(outputs_proto)

초당 쿼리 요청이 많다면 -> TF 서빙을 서버 여러대에 설치 - 쿼리 로드 밸런싱 필요 -> 이를 해결하는 방법 *쿠버네티스*    
많은 서버의 컨테이너를 손쉽게 관리할 수 있음.

하드웨어없이도,    
PaaS 클라우드 플랫폼으로 가상 서버 사용 + 쿠버네티스로 컨테이너 운영 + TF 서빙 설정 + 튜닝, 모니터링


In [2]:
import openai

def generate_poem(prompt):
    # ChatGPT API에 연결하기 위한 인증 정보 설정
    openai.api_key = 'YOUR_API_KEY'

    # ChatGPT API를 통해 시를 생성
    response = openai.Completion.create(
        engine='text-davinci-003',  # GPT-3.5 언어 모델 선택
        prompt=prompt,  # 시 생성을 위한 프롬프트 입력
        max_tokens=100,  # 생성된 시의 최대 토큰 수
        temperature=0.7,  # 다양성 조절을 위한 온도 설정
        n=1,  # 생성할 시의 수
        stop=None,  # 시 생성을 중지할 토큰 설정
        timeout=15,  # 요청 시간 초과 설정
    )

    # 생성된 시 추출
    poem = response.choices[0].text.strip()
    return poem

# 시 생성을 위한 프롬프트 입력
prompt = "봄이 오면\n"
poem = generate_poem(prompt)
print(poem)


봄이 되면 하늘이 푸르고 꽃들이 핑크빛으로 피어나며 나뭇잎이 초록색으로 물들
